In [1]:
import sys
import os
from pathlib import Path

# Add the project root to the python path so it can find 'src'
root_path = Path(os.getcwd()).parent
if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

import pandas as pd
import numpy as np
from src.models.predict import FraudPredictor

RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Paths
TEST_DATA_PATH = "../data/processed/test_identity_final.parquet"
MODEL_PATH = "../artifacts/lightgbm_identity_final.json"
SCHEMA_PATH = "../artifacts/schema/schema_identity_final.json"
STATS_PATH = "../artifacts/schema/reference_stats.json"

# Loading the same predictor used by the API
predictor = FraudPredictor(MODEL_PATH, SCHEMA_PATH, STATS_PATH)
X_test = pd.read_parquet(TEST_DATA_PATH)

In [2]:
results = []

print(f"Starting inference on {len(X_test)} records...")

# For large test sets, we use the predictor.predict logic
# Since our predictor is optimized for single records, we loop or optimize the transform for the whole dataframe.
X_test_transformed = predictor.feature_builder.transform(X_test)

# Final formatting for the model
model_features = predictor.model.feature_name()
X_test_final = X_test_transformed[model_features].copy()

# Apply Categorical types
for col in predictor.cat_cols:
    v = predictor.vocab.get(col, [])
    X_test_final[col] = pd.Categorical(X_test_final[col], categories=v)

# Get raw probabilities
probabilities = predictor.model.predict(X_test_final)

Starting inference on 506691 records...


In [3]:
# Creating the results dataframe
final_outcomes = pd.DataFrame({
    'TransactionAmt': X_test['TransactionAmt'], # Raw amount
    'Probability': probabilities
})

OPTIMAL_THRESHOLD = 0.474

def determine_action(row):
    if row['Probability'] >= OPTIMAL_THRESHOLD:
        return "BLOCK_AND_CHALLENGE"
    elif row['TransactionAmt'] > 2000 and row['Probability'] > 0.10:
        return "MANUAL_REVIEW_REQUIRED"
    else:
        return "APPROVE"

final_outcomes['Action'] = final_outcomes.apply(determine_action, axis=1)


final_outcomes.to_csv("../results/final_test_predictions.csv", index=False)

In [4]:
results_df = pd.read_csv("../results/final_test_predictions.csv")

In [5]:
# Basic Counts
total_transactions = len(results_df)
blocked_count = len(results_df[results_df['Action'] == "BLOCK_AND_CHALLENGE"])
review_count = len(results_df[results_df['Action'] == "MANUAL_REVIEW_REQUIRED"])
approved_count = len(results_df[results_df['Action'] == "APPROVE"])

In [6]:
# High-Value Outlier Analysis ($100k+)
high_value_df = results_df[results_df['TransactionAmt'] > 2000 ]
high_value_flagged = len(high_value_df[high_value_df['Action'] != "APPROVE"])
catch_rate = (high_value_flagged / len(high_value_df)) * 100 if len(high_value_df) > 0 else 0

In [10]:
print(f"--- FINAL INFERENCE REPORT ---")
print(f"Total Transactions Processed: {total_transactions:,}")
print(f"Total Blocked (ML Decision): {blocked_count:,}")
print(f"Total Flagged for Manual Review: {review_count:,}")
print(f"High-Value Catch Rate ($2k+): {catch_rate:.2f}%")

--- FINAL INFERENCE REPORT ---
Total Transactions Processed: 506,691
Total Blocked (ML Decision): 9,649
Total Flagged for Manual Review: 52
High-Value Catch Rate ($2k+): 3.83%


In [9]:
# Check if high-value transactions exist in the test data
high_val_count = len(results_df[results_df['TransactionAmt'] > 2000 ])
max_amt = results_df['TransactionAmt'].max()

print(f"Number of transactions > $2000: {high_val_count}")
print(f"Highest transaction amount in test set: ${max_amt:,.2f}")

Number of transactions > $2000: 1801
Highest transaction amount in test set: $10,270.00
